In [ ]:
# hide

%load_ext nb_black

The nb_black extension is already loaded. To reload it, use:
  %reload_ext nb_black


<IPython.core.display.Javascript object>

In [ ]:
# default_exp django.views

<IPython.core.display.Javascript object>

# Django Views

In [ ]:
# export
from pathlib import Path

from django.http import JsonResponse
from django.http import FileResponse


async def serve_file(request, base, path, num):
    file_path = Path(base) / path / str(num)
    # return JsonResponse({"file_path": file_path})
    return FileResponse(file_path.open("rb"))

<IPython.core.display.Javascript object>

# Export

In [ ]:
# dont_test

from nbdev.export import notebook2script

notebook2script()

Converted 00_core.ipynb.
Converted 01_django_views.ipynb.
Converted 01_fastapi_views.ipynb.
Converted 02_docker_servers.ipynb.
Converted 02_local_servers.ipynb.
Converted 03_benchmark_clients.ipynb.
Converted 04_persistence.ipynb.
Converted 05_run_benchmark.ipynb.
Converted index.ipynb.


<IPython.core.display.Javascript object>